##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Como migrar das APIs Estimator para Keras

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/migrating_estimator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/migrating_estimator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/migrating_estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/migrate/migrating_estimator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
</table>

Este guia demonstra como migrar das APIs `tf.estimator.Estimator` do TensorFlow 1 para as APIs `tf.keras` do TensorFlow 2. Primeiro, você configurará e executará um modelo básico para treinamento e avaliação com `tf.estimator.Estimator`. Em seguida, você executará as etapas equivalentes no TensorFlow 2 com as APIs `tf.keras`. Você também aprenderá como personalizar a etapa de treinamento criando uma subclasse de `tf.keras.Model` e usando `tf.GradientTape`.

- No TensorFlow 1, as APIs `tf.estimator.Estimator` de alto nível permitem que você treine e avalie um modelo, além de realizar inferências e salvar seu modelo (para servir).
- No TensorFlow 2, use as APIs Keras para executar as tarefas mencionadas acima, como [construção de modelo](https://www.tensorflow.org/guide/keras/custom_layers_and_models), aplicação de gradiente, [treinamento](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit), avaliação e previsão.

(Para migrar workflows de salvamento de modelo/checkpoint para o TensorFlow 2, confira os guias de migração [SavedModel](saved_model.ipynb) e [Checkpoint](checkpoint_saved.ipynb).)

## Configuração

Comece com imports e um dataset simples:

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [[0.8], [0.9], [1.]]

## TensorFlow 1: treine e avalie com tf.estimator.Estimator

Este exemplo mostra como realizar treinamento e avaliação com `tf.estimator.Estimator` no TensorFlow 1.

Comece definindo algumas funções: uma função de entrada para os dados de treinamento, uma função de entrada de avaliação para os dados de avaliação e uma função de modelo que informa ao `Estimator` como a operação de treinamento é definida com as características e rótulos:

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

def _eval_input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(1)(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  return tf1.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

Instancie seu `Estimator` e treine o modelo:

In [ ]:
estimator = tf1.estimator.Estimator(model_fn=_model_fn)
estimator.train(_input_fn)

Avalie o programa com o dataset de avaliação:

In [ ]:
estimator.evaluate(_eval_input_fn)

## TensorFlow 2: treine e avalie com os métodos Keras integrados

Este exemplo demonstra como realizar treinamento e avaliação com Keras `Model.fit` e `Model.evaluate` no TensorFlow 2. (Saiba mais no guia [Treinamento e avaliação com métodos integrados](https://www.tensorflow.org/guide/keras/train_and_evaluate)).

- Comece preparando o pipeline do dataset com as APIs `tf.data.Dataset`.
- Defina um modelo Keras [Sequential](https://www.tensorflow.org/guide/keras/sequential_model) simples com uma camada linear (`tf.keras.layers.Dense`).
- Crie uma instância de um otimizador Adagrad (`tf.keras.optimizers.Adagrad`).
- Configure o modelo para treinamento passando a variável `optimizer` e a perda do erro quadrático médio (`"mse"`) para `Model.compile`.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer=optimizer, loss="mse")

Com isto, você está pronto para treinar o modelo chamando `Model.fit`:

In [ ]:
model.fit(dataset)

Finalmente, avalie o modelo com `Model.evaluate`:

In [ ]:
model.evaluate(eval_dataset, return_dict=True)

## TensorFlow 2: treine e avalie com uma etapa de treinamento personalizada e métodos Keras integrados

No TensorFlow 2, você também pode escrever sua própria função de passo de treinamento personalizada com `tf.GradientTape` para executar passos para frente e para trás, enquanto aproveita o suporte de treinamento integrado, como `tf.keras.callbacks.Callback` e `tf.distribute.Strategy`. (Saiba mais em [Personalizando o que acontece no Model.fit](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit) e [Escrevendo loops de treinamento personalizados do zero](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch).)

Neste exemplo, comece criando um `tf.keras.Model` personalizado criando uma subclasse de `tf.keras.Sequential` que substitui `Model.train_step`. (Saiba mais sobre [subclasses tf.keras.Model](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). Dentro dessa classe, defina uma função `train_step` personalizada que, para cada lote de dados, execute um passo para frente e um passo para trás durante uma etapa de treinamento.


In [ ]:
class CustomModel(tf.keras.Sequential):
  """A custom sequential model that overrides `Model.train_step`."""

  def train_step(self, data):
    batch_data, labels = data

    with tf.GradientTape() as tape:
      predictions = self(batch_data, training=True)
      # Compute the loss value (the loss function is configured
      # in `Model.compile`).
      loss = self.compiled_loss(labels, predictions)

    # Compute the gradients of the parameters with respect to the loss.
    gradients = tape.gradient(loss, self.trainable_variables)
    # Perform gradient descent by updating the weights/parameters.
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
    # Update the metrics (includes the metric that tracks the loss).
    self.compiled_metrics.update_state(labels, predictions)
    # Return a dict mapping metric names to the current values.
    return {m.name: m.result() for m in self.metrics}

Em seguida, como antes:

- Prepare o pipeline do dataset com `tf.data.Dataset`.
- Defina um modelo simples com uma camada `tf.keras.layers.Dense`.
- Instancie o Adagrad (`tf.keras.optimizers.Adagrad`)
- Configure o modelo para treinamento com `Model.compile`, enquanto usa o erro quadrático médio (`"mse"`) como função de perda.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

model = CustomModel([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer=optimizer, loss="mse")

Chame `Model.fit` para treinar o modelo:

In [ ]:
model.fit(dataset)

E, finalmente, avalie o programa com `Model.evaluate`:

In [ ]:
model.evaluate(eval_dataset, return_dict=True)

## Próximos passos

Recursos adicionais do Keras que você pode achar úteis:

- Guia: [Treinamento e avaliação com os métodos integrados](https://www.tensorflow.org/guide/keras/train_and_evaluate)
- Guia: [Personalize o que acontece em Model.fit](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit)
- Guia: [Escrevendo um loop de treinamento do zero](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch)
- Guia: [Criando novas camadas e modelos de Keras por meio de subclasses](https://www.tensorflow.org/guide/keras/custom_layers_and_models)

Os guias abaixo podem ajudar na migração de workflows de estratégias de distribuição de APIs `tf.estimator`:

- [Como migrar de TPUEstimator para TPUStrategy](tpu_estimator.ipynb)
- [Como migrar o treinamento de um worker com multiplas GPUs](mirrored_strategy.ipynb)
- [Como migrar o treinamento de múltiplos workers com CPUs/GPUs](multi_worker_cpu_gpu_training.ipynb)